# Test EXAONE
- Created: 2024-12-02 (Mon)
- Updated: 2024-12-02 (Mon)

## Configure

In [4]:
# ModuleNotFoundError: No module named 'torch'
!pip install --quite torch torchvision torchaudio

In [6]:
# ModuleNotFoundError: No module named 'transformers'
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 41.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.0 MB/s eta 0:00:00


In [8]:
# OSError: You are trying to access a gated repo. Make sure to have access to it at https:// ...
!pip install huggingface_hub

## Log into Hugging Face Hub
The Hugging Face token is created and save in `token-to-access-exaone-since-2024-q4.txt`.

Open a new terminal (File > New > Terminal) and run:
```bash
$ huggingface-cli login
# Paste your access token when prompted.
```


In [13]:
# ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`
!pip install accelerate>=0.26.0

In [1]:
# Double-check
import accelerate
print(accelerate.__version__)

1.1.1


In [ ]:
# Restart the kernel; otherwise, the error won't go away.

A significant amount of RAM is required to load the EXAONE-3.0-7.8B-Instruct model.
If RAM is insufficient, a `RuntimeError` is expected. 
```bash
# RuntimeError: unable to mmap 4932636680 bytes from file </home/jupyter/.cache/huggingface/hub/models--LGAI-EXAONE--EXAONE-3.0-7.8B-Instruct/snapshots/7f15baedd46858153d817445aff032f4d6cf4939/model-00001-of-00007.safetensors>: Cannot allocate memory (12)
```

In [4]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           3.6Gi       1.2Gi       262Mi        10Mi       2.1Gi       2.1Gi
Swap:             0B          0B          0B


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)

model.safetensors.index.json:   0%|          | 0.00/23.7k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: unable to mmap 4932636680 bytes from file </home/jupyter/.cache/huggingface/hub/models--LGAI-EXAONE--EXAONE-3.0-7.8B-Instruct/snapshots/7f15baedd46858153d817445aff032f4d6cf4939/model-00001-of-00007.safetensors>: Cannot allocate memory (12)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

In [ ]:
# Choose your prompt
prompt = "Explain who you are"  # English example
prompt = "너의 소원을 말해봐"   # Korean example

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128
)
print(tokenizer.decode(output[0]))